In [2]:
import os
import urllib
import pyodbc
import sqlalchemy
import pandas as pd
import pyreadr
import geopandas as gpd

In [3]:
# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\Dt_df.Rda'
nhts_df = pyreadr.read_r(fname)['Dt_df']

In [4]:
nhts_df['TRPMILES']

rownames
1          0.444444
2          0.444444
3          0.800000
4          4.000000
5          4.000000
            ...    
642288     0.800000
642289    13.600000
642290    12.800000
642291     0.800000
642292    13.600000
Name: TRPMILES, Length: 642292, dtype: float64

In [5]:
nhts_df[nhts_df['TRPTRANS'] == 1]['TRPMILES'].mean()

7.217418054478968

In [75]:
pyreadr.write_rdata(fname, nhts_df, df_name="Dt_df")

# Tours

In [13]:
# Load NHTS survey files
fname = r'C:\Workspace\VisionEval_build\VisionEval\sources\modules\VE2001NHTS\data-raw\ToursByHh_df.Rda'
nhts_df = pyreadr.read_r(fname)['ToursByHh_df']

In [52]:
conn_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=AWS-PROD-SQL\Sockeye; DATABASE=Elmer; trusted_connection=yes"
sql_conn = pyodbc.connect(conn_string)
df_trip = pd.read_sql(sql='select * from HHSurvey.trips_2017_2019', con=sql_conn)
df_hh = pd.read_sql(sql='select * from HHSurvey.households_2017_2019', con=sql_conn)
df_person = pd.read_sql(sql='select * from HHSurvey.persons_2017_2019', con=sql_conn)
df_vehicle = pd.read_sql(sql='select * from HHSurvey.vehicles_2017_2019', con=sql_conn)

In [65]:
nhts_df.columns

Index(['Houseid', 'Distance', 'TravelTime', 'DwellTime', 'StartHome',
       'EndHome', 'Trips', 'Persons', 'Vehid', 'Trptrans', 'Vehtype',
       'HhVehUsed', 'Whyto', 'Disttowk'],
      dtype='object')

In [56]:
# FIXME Since we don't have a tour file in the RSG format, just consider each trip a tour for now
# [print(i) for i in df_trip.columns]
tour_df = df_trip.copy()





# Houseid
# Distance
# TravelTime
# Dwell Time
# Persons
tour_df.rename(columns={'household_id':'Houseid', 'trip_path_distance': 'Distance', 'travtime': 'TravelTime',
                        'activity_duration': 'DwellTime', 'travelers_tot': 'Persons'}, inplace=True)

# StartHome
tour_df[['StartHome','EndHome']] = False
tour_df.loc[tour_df['o_purpose'] == 1, 'StartHome'] = True

# EndHome
tour_df.loc[tour_df['d_purpose'] == 1, 'EndHome'] = True

# Trips
# FIXME: asserting this to be 2 (initial + return trip) since we are not building real tours
tour_df['Trips'] = 2

# Vehid
# Get vehicle ID based on mode_1 values
# 3 - 12 are various household vehilces

# Trptrans

# Vehtype

# HHVehUsed

# Whyto

# Disttowk


In [48]:
tour_df.columns

Index(['Houseid', 'pno', 'day', 'tour', 'jtindex', 'parent', 'subtrs',
       'pdpurp', 'tlvorig', 'tardest', 'tlvdest', 'tarorig', 'toadtyp',
       'tdadtyp', 'topcl', 'totaz', 'tdpcl', 'tdtaz', 'tmodetp', 'tpathtp',
       'tripsh1', 'tripsh2', 'phtindx1', 'phtindx2', 'fhtindx1', 'fhtindx2',
       'toexpfac', 'id', 'c', 'd', 't', 'tautotime', 'tautocost', 'tautodist',
       'hhsize', 'hhvehs', 'hhwkrs', 'hhftw', 'hhptw', 'hhret', 'hhoad',
       'hhuni', 'hhhsc', 'hh515', 'hhcu5', 'hhincome', 'hownrent', 'hrestype',
       'hhtaz', 'hhparcel', 'hhexpfac', 'samptype', 'DwellTime', 'StartHome',
       'EndHome', 'Trips'],
      dtype='object')

In [34]:
tour_df.DwellTime.mean()

263.1086880364337

In [30]:
tour_df[tour_df['DwellTime'] < 0][['tlvdest','tardest']]

,tlvdest,tardest
0,826,1100
1,579,1101
2,716,1099
3,936,1287
4,1103,1321
...,...,...
20375,320,930
20376,775,875
20377,935,970
20378,300,960
